In [2]:
!pip install langchain langchainhub transformers sentencepiece huggingface_hub

In [3]:
!pip install accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 61.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 50.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 79.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [4]:
!pip install langchain-huggingface
!pip install langchain-community langchain-core

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.1 MB/s eta 0:00:00


In [5]:
!pip install --upgrade langchain huggingface_hub

In [6]:
!pip install langchain_groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.7/126.7 kB 3.3 MB/s eta 0:00:00


In [27]:
from langchain.agents import initialize_agent, Tool, AgentType
from langchain_groq import ChatGroq
from langchain.tools import tool
from langchain.memory import ConversationBufferMemory
import os
from getpass import getpass

# For Google Colab environment
from google.colab import userdata
os.environ["GROQ_API_KEY"] = userdata.get('groq-apikey')

# Mock database for writing tips
writing_tips = {
    "thesis": "A thesis statement should clearly express the main idea of your paper in one sentence.",
    "cohesion": "Use transition words like 'however', 'therefore', and 'moreover' to maintain flow between paragraphs.",
    "introduction": "An introduction should provide background information and introduce the main argument or purpose of the paper.",
    "conclusion": "A conclusion should summarize the key points discussed and restate the thesis in light of the evidence presented.",
    "clarity": "Write clearly and concisely, avoiding unnecessary jargon or complex sentences that might confuse the reader.",
    "evidence": "Support your claims with credible sources and data, ensuring that each piece of evidence is relevant to your argument.",
    "audience": "Consider your audience’s knowledge and expectations when writing, adjusting your tone and language accordingly.",
    "citation": "Properly cite all your sources using the appropriate citation style (APA, MLA, Chicago, etc.) to avoid plagiarism.",
    "structure": "Ensure that your paper has a clear and logical structure, with well-organized paragraphs that each focus on a single idea.",
    "editing": "Always edit your paper for grammar, punctuation, and spelling errors. A well-polished paper leaves a strong impression.",
    "revision": "Revise your paper after feedback, making improvements to both the content and structure based on the reviewer’s comments."
}


# List of supported languages for rewrite tool
possible_languages = [
    "english", "spanish", "german",
    "hindi", "bengali", "chinese", "japanese", "korean",
    "urdu", "portuguese", "italian", "russian"
]

@tool
def get_writing_tips(query: str) -> str:
    """Extracts a topic from the user query and returns a writing tip from the knowledge base."""
    keywords = writing_tips.keys()
    query_lower = query.strip().lower()

    for key in keywords:
        if key in query_lower:
            return writing_tips[key]

    return "❌ Sorry, I couldn't find any tip related to your query in our database."

@tool
def text_rewrite_multilingual(query: str) -> str:
    """Prompt the LLM to translate the given text to the specified language if found in the query."""
    global possible_languages
    lower_query = query.lower()

    # Strip the language from the query to prevent repetition
    for lang in possible_languages:
        if lang in lower_query:
            # Extract the text by removing the language part from the query
            query_text = query.replace(lang, '').strip()

            # Return the translation prompt without extra language info
            return f"✅ Please translate the following text into {lang.capitalize()}:\n{query_text}"


# LLM setup
llm = ChatGroq(model="llama3-8b-8192", temperature=0.7)

# Tool setup
tools = [
    Tool.from_function(
        func=get_writing_tips,
        name="get_writing_tip",
        description="Useful for retrieving writing tips about structure, thesis, or cohesion."
    ),
    Tool.from_function(
        func=text_rewrite_multilingual,
        name="text_rewrite_multilingual",
        description="Useful for rewriting text in another language like Arabic, Spanish, etc."
    )
]

# Memory for chat
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# Initialize the agent
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
    memory=memory,
    verbose=True,
)

# Interactive chat loop
print("👋 Hey! I'm your writing assistant. Ask me anything related to writing tips or text rewriting.\nType 'exit' to quit.\n")

while True:
    query = input("You: ")
    if query.lower() in ["exit", "quit"]:
        print("👋 Goodbye!")
        break

    # Filter out unrelated queries
    if not any(k in query.lower() for k in writing_tips.keys()) and not any(lang in query.lower() for lang in possible_languages):
        print("❌ I can only assist with writing tips (like 'thesis', 'cohesion', or 'clarity') or rewrite text in another language (e.g., 'Arabic', 'Spanish', etc.). Please make sure your query matches one of these topics.")

        continue

    try:
        response = agent.run(f"You are a friendly writing assistant. Only use tools when necessary. \
Only use the `get_writing_tip` tool if a topic matches the keys in the writing_tips dict. \
Only use `text_rewrite_multilingual` to rewrite text in a language specified in the input. \
Never guess or generate content on your own. User's input: {query}")
        print(f"Assistant: {response}\n")
    except Exception as e:
        print(f"⚠️ Error: {e}")


👋 Hey! I'm your writing assistant. Ask me anything related to writing tips or text rewriting.
Type 'exit' to quit.

You: rewrite this text "i love cake" in bangla
❌ I can only assist with writing tips (like 'thesis', 'cohesion', or 'clarity') or rewrite text in another language (e.g., 'Arabic', 'Spanish', etc.). Please make sure your query matches one of these topics.
You: rewrite this text "i love cake" in bengali


> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: text_rewrite_multilingual
Action Input: "i love cake", Bengali
Observation: ✅ Please translate the following text into Bengali:
i love cake", Bengali
Thought:Thought: Do I need to use a tool? No
AI: আমি কেকটি ভাল করি (I love cake)

> Finished chain.
Assistant: আমি কেকটি ভাল করি (I love cake)

You: give me a writing tip on thesis 


> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: get_writing_tip
Action Input: thesis
Observation: A thesis statement should c